In [ ]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 4.1 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 16.6 MB 19.5 MB/s 
     |████████████████████████████████| 73 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 42.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [ ]:
# -*- coding: utf-8 -*-

import codecs
import _pickle as cPickle
import random
from collections import defaultdict
import pandas as pd

#set up output file
fout_src_train = codecs.open('src_train.txt','wb','utf-8')
fout_tgt_train = codecs.open('tgt_train.txt','wb','utf-8')
fout_src_valid = codecs.open('src_valid.txt','wb','utf-8')
fout_tgt_valid = codecs.open('tgt_valid.txt','wb','utf-8')
fout_src_test = codecs.open('src_test.txt','wb','utf-8')
fout_tgt_test = codecs.open('tgt_test.txt','wb','utf-8')

#read in  data
fin = codecs.open('english_merged.txt','rb','utf-8')

sources = []
targets = []

for line in fin:
	parts = line.strip().split()
	lemma = parts[2]
	form = parts[3]
	sources.append(' '.join(lemma))
	targets.append(' '.join(form))
fin.close()

pairs = list(zip(sources,targets))
random.shuffle(pairs)

#split into train and test
train = pairs[:int(.8*len(pairs))]
valid = pairs[int(.8*len(pairs)):int(.9*len(pairs))]
test = pairs[int(.9*len(pairs)):]

#write the outputs
for s,t in train:
	fout_src_train.write(s + '\n')
	fout_tgt_train.write(t + '\n')

for s,t in valid:
	fout_src_valid.write(s + '\n')
	fout_tgt_valid.write(t + '\n')

for s,t in test:
	fout_src_test.write(s + '\n')
	fout_tgt_test.write(t + '\n')



fout_src_train.close()
fout_tgt_train.close()
fout_src_valid.close()
fout_tgt_valid.close()
fout_src_test.close()
fout_tgt_test.close()


In [ ]:
!onmt_build_vocab -config config-exp1.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-19 12:32:13,982 INFO] Counter vocab from 10000 samples.
[2022-03-19 12:32:13,982 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-03-19 12:32:13,995 INFO] corpus_1's transforms: TransformPipe()
[2022-03-19 12:32:14,099 INFO] Counters src:38
[2022-03-19 12:32:14,100 INFO] Counters tgt:38


In [ ]:
!onmt_train -config config-exp1.yaml

[2022-03-19 12:32:19,462 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-19 12:32:19,462 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-19 12:32:19,462 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-19 12:32:19,462 INFO] Parsed 2 corpora from -data.
[2022-03-19 12:32:19,462 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-19 12:32:19,463 INFO] Loading vocab from text file...
[2022-03-19 12:32:19,463 INFO] Loading src vocabulary from experiment_1/run/example.vocab.src
[2022-03-19 12:32:19,463 INFO] Loaded src vocab has 38 tokens.
[2022-03-19 12:32:19,463 INFO] Loading tgt vocabulary from experiment_1/run/example.vocab.tgt
[2022-03-19 12:32:19,464 INFO] Loaded tgt vocab has 38 tokens.
[2022-03-19 12:32:19,464 INFO] Building fields with vocab in counters...
[2022-03-19 12:32:19,464 INFO]  * tgt vocab size: 42.
[2022-03-19 12:32:19,465 INFO]  * src vocab size

In [ ]:
!onmt_translate -model experiment_1/run/model_step_1000.pt -src experiment_1/src_test.txt -output experiment_1/test-decoded.txt -gpu 0 -verbose -beam_size 12 


[2022-03-19 12:36:48,658 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-03-19 12:36:48,775 INFO] 
SENT 1: ['r', '@', 'U', 'm', '&', 'n', 't', '@', 's', 'a', 'I', 'z']
PRED 1: r @ U m & n t E s a I z d
PRED SCORE: -1.0747

[2022-03-19 12:36:48,776 INFO] 
SENT 2: ['s', 't', 'V', 'f']
PRED 2: s t V f t
PRED SCORE: -0.0659

[2022-03-19 12:36:48,776 INFO] 
SENT 3: ['r', 'i', ':', 'p', 'l', 'e', 'I', 's']
PRED 3: r i : p l e I s t
PRED SCORE: -0.0971

[2022-03-19 12:36:48,776 INFO] 
SENT 4: ['l', '&', 'D', '@', 'r'

In [ ]:
# -*- coding: utf-8 -*-
"""Match train/test split from experiment 1, by lemma."""
import codecs
import _pickle as cPickle
import random
from collections import defaultdict
import pandas as pd

#set up output file
fout_src_train = codecs.open('src_train_tagged.txt','wb','utf-8')
fout_tgt_train = codecs.open('tgt_train_tagged.txt','wb','utf-8')
fout_src_valid = codecs.open('src_valid_tagged.txt','wb','utf-8')
fout_src_test = codecs.open('src_test_tagged.txt','wb','utf-8')

#modify every line in the current valid data
fin = codecs.open('experiment_1/src_valid.txt','rb','utf-8')
for line in fin:
	fout_src_valid.write('<V;PST> ' + line)
fin.close()
#modify every line in the current test data
fin = codecs.open('experiment_1/src_test.txt','rb','utf-8')
for line in fin:
	fout_src_test.write('<V;PST> ' + line)
fin.close()

#read in a set of valid lemmas from the current train data
ok_lemmas = set()
fin =  codecs.open('experiment_1/src_train.txt','rb','utf-8')
for line in fin:
	ok_lemmas.add(line.strip())
fin.close()

In [ ]:
!onmt_build_vocab -config config-exp2.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
path_tgt is None, it should be set unless the task is language modeling
path_tgt is None, it should be set unless the task is language modeling
Corpus test's weight should be given. We default it to 1 for you.
[2022-03-19 12:49:59,632 INFO] Counter vocab from 10000 samples.
[2022-03-19 12:49:59,632 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-03-19 12:49:59,644 INFO] corpus_1's transforms: TransformPipe()
[2022-03-19 12:49:59,645 INFO] test's transforms: TransformPipe()
[2022-03-19 12:49:59,750 INFO] Counters src:39
[2022-03-19 12:49:59,751 INFO] Counters tgt:39


In [ ]:
!onmt_train -config config-exp2.yaml

[2022-03-19 12:55:48,358 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-19 12:55:48,358 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-19 12:55:48,358 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-19 12:55:48,358 WARNING] path_tgt is None, it should be set unless the task is language modeling
[2022-03-19 12:55:48,358 INFO] Missing transforms field for test data, set to default: [].
[2022-03-19 12:55:48,359 WARNING] path_tgt is None, it should be set unless the task is language modeling
[2022-03-19 12:55:48,359 WARNING] Corpus test's weight should be given. We default it to 1 for you.
[2022-03-19 12:55:48,359 INFO] Parsed 3 corpora from -data.
[2022-03-19 12:55:48,359 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-19 12:55:48,359 INFO] Loading vocab from text file...
[2022-03-19 12:55:48,359 INFO] Loading src vocabulary from experiment_2/run/example.vo

In [ ]:
!onmt_translate -model experiment_2/run/model_step_1000.pt -src experiment_1/src_test.txt -output experiment_1/test-decoded.txt -gpu 0 -verbose -beam_size 12 

Traceback (most recent call last):
  File "/usr/local/bin/onmt_translate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/translate.py", line 54, in main
    translate(opt)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/translate.py", line 16, in translate
    translator = build_translator(opt, logger=logger, report_score=True)
  File "/usr/local/lib/python3.7/dist-packages/onmt/translate/translator.py", line 32, in build_translator
    fields, model, model_opt = load_test_model(opt)
  File "/usr/local/lib/python3.7/dist-packages/onmt/model_builder.py", line 85, in load_test_model
    map_location=lambda storage, loc: storage)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 594, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/li

In [ ]:
!zip -r /content/NLP.zip /content

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/.feature_flags_config.yaml (deflated 23%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.metricsUUID (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2022.03.09/ (stored 0%)
  adding: content/.config/logs/2022.03.09/14.47.22.983659.log (deflated 54%)
  adding: content/.config/logs/2022.03.09/14.47.03.286667.log (deflated 91%)
  adding: content/.config/logs/2022.03.09/14.47.40.126263.log (deflated 86%)
  adding: content